# State optimization in LMG model (multiparameter) 

This is the source codes for the example discussed in Sec. V in Ref. [1].

The Hamiltonian of the Lipkin–Meshkov–Glick (LMG) model is
\begin{align}
H_{\mathrm{LMG}}=-\frac{\lambda}{N}(J_1^2+gJ_2^2)-hJ_3,
\end{align}

where $N$ is the number of spins of the system, $\lambda$ is the spin–spin interaction strength, $h$ is the strength of the 
external field and $g$ is the anisotropic parameter. $J_i=\frac{1}{2}\sum_{j=1}^N \sigma_i^{(j)}$ ($i=1,2,3$) is the collective spin operator with $\sigma_i^{(j)}$ the $i$th Pauli matrix for the $j$th spin. In multi-parameter scenario, we take $g$ and $h$ as the unknown parameters to be estimated. The states are expanded as $|\psi\rangle=\sum^J_{m=-J}c_m|J,m\rangle$ with $|J,m\rangle$ the Dicke state and $c_m$ a complex coefficient. Here we fixed $J=N/2$. In this example, the probe state is optimized for both noiseless scenario and collective dephasing noise. The dynamics under collective dephasing can be expressed as
\begin{align}
\partial_t\rho = -i[H_{\mathrm{LMG}},\rho]+\gamma \left(J_3\rho J_3-\frac{1}{2}\left\{\rho, J^2_3\right\}\right)
\end{align}

with $\gamma$ the decay rate.

In this case, all searches with different algorithms start from the coherent spin state defined by
$|\theta=\frac{\pi}{2},\phi=\frac{\pi}{2}\rangle=\exp(-\frac{\theta}{2}e^{-i\phi}J_{+}+\frac{\theta}{2}e^{i\phi}J_{-})|J,J\rangle$ with $J_{\pm}=J_1{\pm}iJ_2$. The performance of optimal states obtained via different weight matrices be different, here we show the state optimization when $W=\mathrm{diag}(1/3,2/3)$.

[1] M. Zhang et al., QuanEstimation: an open-source toolkit for quantum parameter estimation,
arXiv:2022.xxxx.

In [ ]:
from quanestimation import *
import numpy as np
from qutip import *

N = 8
# generation of coherent spin state
psi_css = spin_coherent(0.5*N, 0.5*np.pi, 0.5*np.pi, type="ket").full()
psi_css = psi_css.reshape(1, -1)[0]
# guessed state
psi0 = [psi_css]
# free Hamiltonian
Lambda = 1.0
g = 0.5
h = 0.1
Jx, Jy, Jz = jmat(0.5 * N)
Jx, Jy, Jz = Jx.full(), Jy.full(), Jz.full()
H0 = -Lambda*(np.dot(Jx, Jx) + g*np.dot(Jy, Jy))/N - h*Jz
# derivatives of the free Hamiltonian on the g and h
dH = [-Lambda * np.dot(Jy, Jy)/N, -Jz]
# dissipation
decay = [[Jz, 0.1]]
# time length for the evolution
tspan = np.linspace(0.0, 10.0, 2500)
# weight matrix
W = np.array([[1/3, 0.0], [0.0, 2/3]])

State optimization algorithm: AD

In [ ]:
AD_paras = {"Adam":False, "psi0":psi0, "max_episode":300, "epsilon":0.01, "beta1":0.90, "beta2":0.99}
state = StateOpt(savefile=False, method="AD", **AD_paras)
state.dynamics(tspan, H0, dH, decay=decay)
# choose QFIM as the objective function
state.QFIM(W=W)

State optimization algorithm: PSO

In [ ]:
PSO_paras = {"particle_num":10, "psi0":psi0, "max_episode":[1000, 100], "c0":1.0, "c1":2.0, "c2":2.0, "seed":1234}
state = StateOpt(savefile=False, method="PSO", **PSO_paras)
state.dynamics(tspan, H0, dH, decay=decay)
# choose QFIM as the objective function
state.QFIM(W=W)

State optimization algorithm: DE

In [ ]:
DE_paras = {"popsize":10, "psi0":psi0, "max_episode":1000, "c":1.0, "cr":0.5, "seed":1234}
state = StateOpt(savefile=False, method="DE", **DE_paras)
state.dynamics(tspan, H0, dH, decay=decay)
# choose QFIM as the objective function
state.QFIM(W=W)

State optimization algorithm: DDPG

In [ ]:
DDPG_paras = {"layer_num":4, "layer_dim":250, "max_episode":500, "seed":1234}
state = StateOpt(savefile=False, method="DDPG", **DDPG_paras)
state.dynamics(tspan, H0, dH, decay=decay)
# choose QFIM as the objective function
state.QFIM(W=W)

State optimization algorithm: NM

In [ ]:
NM_paras = {"state_num":20, "psi0":psi0, "max_episode":1000, "ar":1.0, "ae":2.0, "ac":0.5, "as0":0.5, "seed":1234}
state = StateOpt(savefile=False, method="NM", **NM_paras)
state.dynamics(tspan, H0, dH, decay=decay)
# choose QFIM as the objective function
state.QFIM(W=W)